This unit is to read the air temperature 
as well as log GPS (bluefly) and humidity to the SD card

We can get all the selected components working and producing 
same file format as the original one.  And then solder together

Then we need to design a 3D-printed housing for it that rigidly 
attaches to an upright profile

Then take it out and run around mapping warm and cold areas 

In [368]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [13]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython/drivers/sdcard/sdcard.py /lib/
%sendtofile --source ../Sensor_Kennel/SI7021_funcs.py

Sent 272 lines (8218 bytes) to /lib/sdcard.py.


In [366]:
%sendtofile main.py

from machine import Pin, SPI, UART, I2C
import sdcard, os, time

i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
spi = SPI(sck=Pin(5), mosi=Pin(22), miso=Pin(27))  # should be mosi=Pin19 but is broke on screenless heltec
uart = UART(1, baudrate=57600, rx=13, tx=17)
sd = sdcard.SDCard(spi, Pin(23))
os.mount(sd, '/sd')

sdfile = None
def opennextlogfileforwriting(mstamp):
    global sdfile
    if not sum((f[0]=="LOG")  for f in os.ilistdir("sd")):
        os.mkdir("sd/LOG")
    fnum = 1+max((int(f[0][:3])  for f in os.ilistdir("sd/LOG")  if f[3]>10), default=0)
    fname = "sd/LOG/{:03d}.TXT".format(fnum)
    print("Opening file", fname)
    sdfile = open(fname, "w")
    sdfile.write("Logfile: {}".format(fname))
    sdfile.write("Device number: 3")
    sdfile.write("Rt[ms]d\"[isodate]\"e[latdE]n[latdN]f[lngdE]o[lngdN] GPS cooeffs\n") 
    sdfile.write("Qt[ms]u[ms midnight]y[lat600000]x[lng600000]a[alt] GPS\n") 
    sdfile.write("Vt[ms]v[kph100]d[deg100] GPS velocity\n") 
    sdfile.write("Ft[ms]p[milibars] bluefly pressure\n") 
    sdfile.write("Gt[ms]r[rawhumid]a[rawtemp] si7021Humidity meter\n") 
    sdfile.write("Nt[ms]r[rawadc]s[resistance] nickel wire sensor\n") 

opennextlogfileforwriting(time.ticks_ms())

from SI7021_funcs import setupSI7021, SI7021checkchip, SI7021printstatus, SI7021humiditytemp, SI7021humiditytempBin
setupSI7021(i2c)
SI7021checkchip()

mstampSI7021 = 0
bh = bytearray(2)
bt = bytearray(2)

def calchumidtemps():
    rh = (bh[0]<<8) + (bh[1]&0xFC)
    rt = (bt[0]<<8) + (bt[1]&0xFC)
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

def readlogSI7021():
    global mstampSI7021
    mstamp = time.ticks_ms()
    if mstampSI7021 == 0:
        i2c.writeto(0x40, b'\xE5')
        mstampSI7021 = mstamp + 20
    elif mstamp > mstampSI7021:
        i2c.readfrom_into(0x40, bh)
        i2c.readfrom_mem_into(0x40, 0xE0, bt)
        if sdfile:
            sdfile.write("Ft{:08X}r{:02X}{:02X}a{:02X}{:02X}\n".format(mstamp, bh[0], bh[1], bt[0], bt[1]))
        else:
            print(mstampSI7021, calchumidtemps())
        mstampSI7021 = 0

pr = 0
def readlogbluefly():
    global pr
    b = uart.readline()
    if not b:  
        return
    mstamp = time.ticks_ms()
    if b[:3] == b'PRS':
        pr = int(b[4:-2], 16)
        sdfile.write("Ft{:08X}p{:06X}\n".format(mstamp, pr))
    if b[:1] == b'$':
        sdfile.write("{}\n".format(b.decode().strip()))
        print(b)

prevtstamp = 0
while True:
    tstamp = time.ticks_ms()
    readlogSI7021()
    readlogbluefly()
    if tstamp > prevtstamp + 2000:
        sdfile.flush()
        prevtstamp = tstamp

Sent 79 lines (2654 bytes) to main.py.


In [367]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-603-gfb7dabb9 on 2018-05-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f2146a97d68, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [372]:
%fetchfile --binary sd/LOG/001.TXT

Fetched 69930=69930 bytes from sd/LOG/001.TXT.
Saving file to '001.TXT'

In [370]:
import os
print(os.listdir("sd/LOG"))



['001.TXT']


In [171]:
# The full implementation of the reading and logging of SI7021 data
import time
mstampSI7021 = 0
bh = bytearray(2)
bt = bytearray(2)

def calchumidtemps():
    rh = (bh[0]<<8) + (bh[1]&0xFC)
    rt = (bt[0]<<8) + (bt[1]&0xFC)
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

def readlogSI7021():
    global mstampSI7021
    mstamp = time.ticks_ms()
    if mstampSI7021 == 0:
        i2c.writeto(0x40, b'\xE5')
        mstampSI7021 = mstamp + 20
    elif mstamp > mstampSI7021:
        i2c.readfrom_into(0x40, bh)
        i2c.readfrom_mem_into(0x40, 0xE0, bt)
        if sdfile:
            sdfile.write("Ft{:08X}r{:02X}{:02X}a{:02X}{:02X}\n".format(mstamp, bh[0], bh[1], bt[0], bt[1]))
        else:
            print(mstampSI7021, calchumidtemps())
        mstampSI7021 = 0

        
#def loggpsabs(mstamp, mstampmidnight, latminutes10000, lngminutes10000, altitude10, bbadgpschecksum, devno):
#    sdfile.write("Qt{:08X}u{:08X}y{:08X}x{:08X}a{:04X}A{:1X}\n".format(mstamp, mstampmidnight, latminutes10000, lngminutes10000, altitude10, devno))
#def loggpsabs(mstamp, velkph100, veldegrees100, devno):
#    sdfile.write("Vt{:08X}v{:04X}d{:06X}A{:1X}\n".format(mstamp, mstampmidnight, latminutes10000, lngminutes10000, altitude10, bbadgpschecksum, devno))
    
pr = 0
def readlogbluefly():
    global pr
    b = uart.readline()
    if not b:  return
    mstamp = time.ticks_ms():
    if b[:3] == b'PRS':
        pr = int(b[4:-2], 16)
        sdfile.write("Ft{:08X}p{:06X}\n".format(mstamp, pr))
    if b[:1] == b'$':
        sdfile.write("{}\n".format(b.decode().strip()))


In [ ]:
# Nickel wire setup code
import machine, time
x = machine.ADC(machine.Pin(36))
x.atten(x.ATTN_11DB)
x.width(x.WIDTH_12BIT)

fout = open("data4.txt", "w")
for i in range(6000):
    fout.write("%d,%d\n" % (time.ticks_ms(), x.read()))
    if (i%1000) == 0:
        print(i)
fout.close()


In [215]:
# Bluefly work Tx->13 Rx->17
import machine
uart = machine.UART(1, baudrate=57600, rx=13, tx=17)


In [334]:
for i in range(1000):
    b = uart.readline()
    if b:
        print(b)
        if b[:3] != b"PRS":
            break

b'PRS 18B1D\r\n'
b'PRS 18B1D\r\n'
b'PRS 18B1E\r\n'
b'PRS 18B1C\r\n'
b'PRS 18B1B\r\n'
b'PRS 18B1F\r\n'
b'PRS 18B1E\r\n'
b'PRS 18B1D\r\n'
b'PRS 18B1D\r\n'
b'PRS 18B1C\r\n'
b'PRS 18B1B\r\n'
b'PRS 18B1E\r\n'
b'PRS 18B1D\r\n'
b'PRS 18B1E\r\n'
b'$GPGGA,000000.800,,,,,0,0,,,M,,M,,*40\r\n'


In [ ]:
    // $GPGGA,064306.590,5323.7234,N,00258.3325,W,  2, 10, 1.00,38.3,M,49.4,M,0000,0000*4A
    positionfixindicator = recline[recblock[6]] - '0'; // is this valid case?
    if (positionfixindicator == 0)
        return false; 

    SetIsoTimestampFromGps(recline + recblock[1], 0);   // no date included
    latminutes10000 = minutes10000parse(recline + recblock[2], false)*(recline[recblock[3]] == 'N' ? 1 : -1); 
    lngminutes10000 = minutes10000parse(recline + recblock[4], true)*(recline[recblock[5]] == 'E' ? 1 : -1); 
    //nsatellitesused = intparse(recline + recblock[7]); 
    horizontaldilutionofposition100 = float10parse(recline + recblock[8], 100); 


In [338]:
print(b.decode().split(","))
print(b.decode().strip())


['$GPGGA', '000000.800', '', '', '', '', '0', '0', '', '', 'M', '', 'M', '', '*40\r\n']
$GPGGA,000000.800,,,,,0,0,,,M,,M,,*40


In [ ]:
bool BlueflyGpsData::IsGpvtg() { return ((strcmp(recline, "$GPVTG") == 0) && (nrecblock == 10)); }
bool BlueflyGpsData::ParseGpvtg()
{
    // $GPVTG,62.47,T,,M,0.29,N,0.53,K,D*02
    if (recline[recblock[9]] == 'N')  // undocumented but observed
        return false; 
    veldegrees100 = float100parse(recline + recblock[1]); 
    //assert strcmp(recline + recblock[2], "T"); 
    //assert strcmp(recline + recblock[4], "M"); 
    //velknots100 = float100parse(recline + recblock[5]); 
    //assert strcmp(recline + recblock[6], "N"); 
    velkph100 = float100parse(recline + recblock[7]); 


b'PRS 18B1D\r\n'


In [263]:
help(u)

object bytearray(b'S 18B1A\r\nPRS 18B19\r\nPRS 18B19\r\nPRS 18B1B\r\n8B1B\r\nPRS 18B1D\r\nP') is of type bytearray
  append -- <function>
  extend -- <function>


In [ ]:
def loggpsabs(mstamp, mstampmidnight, latminutes10000, lngminutes10000, altitude10, bbadgpschecksum, devno):
    sdfile.write("Qt{:08X}u{:08X}y{:08X}x{:08X}a{:04X}A{:1X}\n".format(mstamp, mstampmidnight, latminutes10000, lngminutes10000, altitude10, devno))

def loggpsabs(mstamp, velkph100, veldegrees100, devno):
    sdfile.write("Vt{:08X}v{:04X}d{:06X}A{:1X}\n".format(mstamp, mstampmidnight, latminutes10000, lngminutes10000, altitude10, bbadgpschecksum, devno))
                 
def logbfprs(mstamp, prsreading):
    sdfile.write("Ft{:08X}p{:06X}\n".format(mstamp, prsreading))
